# Saving data to excel
- categories: [Python, pandas, openpyxl, excel]

Perviously (Data Extraction with Python <a href='https://tikuischan.github.io/fastpages_blog/python/dataextraction/pandas/excel/2021/04/04/Data-Extraction-with-Python-Part2.html'>part 2</a> and <a href='https://tikuischan.github.io/fastpages_blog/python/dataextraction/openpyxl/excel/2021/04/16/Data-Extraction-with-Python-Part3.html'>part 3</a>) we've learnt how to extract data from excel files. Sometimes it is easier for our client (especially when they are not familiar to programming) to check the data if we provide it in excel form. The simpliest way to do it is using <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html'>DataFrame().to_excel</a> function in pandas. 

In [1]:
import pandas as pd


sample_data = {'col_1': [1, 2, 3], 'col_2': ['a', 'b', 'c'], 'col_3': [4, 5, 6]}
sample_df = pd.DataFrame(sample_data)
sample_df

,col_1,col_2,col_3
0,1,a,4
1,2,b,5
2,3,c,6


In [ ]:
# simply input file name and we can find the file in the same folder of the python file
sample_df.to_excel('sample_df.xlsx', engine='openpyxl')

# we can also put path-like format to save the excel file in specific folder
sample_df.to_excel('home/target_folder/sample_df.xlsx')

In pratice sometimes things get a bit more complicated. For example there are mulitple tables we need to put in a single excel, or for whatever reason we need to export data in batch, and we want to put all data in a single file, then we need a bit extra work.

For assigning sheet name, we can simply use the parameter sheet_name.

In [2]:
sample_df.to_excel('sample_df.xlsx', sheet_name='sample_df')
sample_df.to_excel('sample_df.xlsx', sheet_name='sample_df2')
pd.read_excel('sample_df.xlsx')

,Unnamed: 0,col_1,col_2,col_3
0,0,1,a,4
1,1,2,b,5
2,2,3,c,6


But as we can see in the output file, there is only one sheet named "sample_df2" in the excel file, pandas will create a new file and remove the file with the same name in this case. To tell pandas to write both sheets in the same file, we can use a ExcelWriter object to lock on a single file, and pass to to_excel to write on the same file.

In [3]:
writer = pd.ExcelWriter('sample_df.xlsx')
sample_df.to_excel(writer, sheet_name='sample_df')
sample_df.to_excel(writer, sheet_name='sample_df2')
writer.save()

If the file is already exist, we can add "mode='a'" to avoid losing all the existing data.

In [4]:
writer = pd.ExcelWriter('sample_df.xlsx', mode='a')
sample_df.to_excel(writer, sheet_name='sample_df3')
sample_df.to_excel(writer, sheet_name='sample_df4')
writer.save()

What if we have multiple tables with the same columns and want to concatenate them into same file? There are two ways to do that, the simple way is to combine the tables before write to excel; and the other is to get the maximum row number of the sheet using load_workbook of openpyxl, and feed to "start_row" of "to_excel".

In [5]:
from openpyxl import load_workbook


table = 'sample_df'
writer = pd.ExcelWriter('sample_df.xlsx', mode='a')
startrow = 0

# try to open an existing workbook
writer.book = load_workbook('sample_df.xlsx')

# get the last row in the existing Excel sheet
if startrow == 0 and table in writer.book.sheetnames:
    # the last row with data is recorded in max_row
    startrow = writer.book[table].max_row
    print(f'the last row in {table} is {startrow}')

writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
sample_df.to_excel(writer, sheet_name=table, startrow=startrow)
writer.save()

pd.read_excel('sample_df.xlsx')

the last row in sample_df is 4


,Unnamed: 0,col_1,col_2,col_3
0,0.0,1,a,4
1,1.0,2,b,5
2,2.0,3,c,6
3,NaN,col_1,col_2,col_3
4,0.0,1,a,4
5,1.0,2,b,5
6,2.0,3,c,6


Now we have successfully append data after the last row, the only problem is the script above will also include the column name as a row and get rid of the meaningless "Unnamed: 0" column. we can simply use the "header" and "index" parameters to achieve them.

In [6]:
table = 'sample_df'
writer = pd.ExcelWriter('sample_df.xlsx', mode='a')
startrow = 0
header = True

# try to open an existing workbook
writer.book = load_workbook('sample_df.xlsx')

# get the last row in the existing Excel sheet
if startrow == 0 and table in writer.book.sheetnames:
    # the last row with data is recorded in max_row
    startrow = writer.book[table].max_row
    print(f'the last row in {table} is {startrow}')

if startrow > 0:
    header = False

# copy existing sheets 
writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
sample_df.to_excel(writer, sheet_name=table, startrow=startrow, header=header)
writer.save()

pd.read_excel('sample_df.xlsx')

the last row in sample_df is 8


,Unnamed: 0,col_1,col_2,col_3
0,0.0,1,a,4
1,1.0,2,b,5
2,2.0,3,c,6
3,NaN,col_1,col_2,col_3
4,0.0,1,a,4
5,1.0,2,b,5
6,2.0,3,c,6
7,0.0,1,a,4
8,1.0,2,b,5
9,2.0,3,c,6


In [8]:
sample_df.to_excel('sample_df2.xlsx', sheet_name='sample_df', index=False)
pd.read_excel('sample_df2.xlsx')

,col_1,col_2,col_3
0,1,a,4
1,2,b,5
2,3,c,6


Combine everything together, we can write a simple function like that:

In [23]:
import os


# assume the data are in dict form: {table_name1: DataFrame1, ...}
def export2excel(dict_form_data, file_path):
    if os.path.exists(file_path):
        load_wb = True
        writer = pd.ExcelWriter(file_path, mode='a')
    else:
        load_wb = False
        writer = pd.ExcelWriter(file_path)
        
    # for loop in dict loop through the keys
    for table in dict_form_data:
        startrow = 0
        header = True
        if load_wb:
            try:
                # try to open an existing workbook
                writer.book = load_workbook(file_path)

                # get the last row in the existing Excel sheet
                if startrow == 0 and table in writer.book.sheetnames:
                    startrow = writer.book[table].max_row

                writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
            except FileNotFoundError:
                pass
        
        if startrow > 0:
            header = False

        dict_form_data[table].to_excel(writer, sheet_name=table, startrow=startrow, index=False, header=header)
        writer.save()
    print(f'finish exploting to {file_path}')
    return

The function will create a new file if "file_path" not exist.

In [25]:
data = {'sample_sheet_1': sample_df, 'sample_sheet_2': sample_df, 'sample_sheet_3': sample_df}
export2excel(data, 'final_sample.xlsx')

finish exploting to final_sample.xlsx


Run again and now we can see the function actually append data to corresponding sheets.

In [26]:
export2excel(data, 'final_sample.xlsx')

finish exploting to final_sample.xlsx


For a more complete function, here is a reference from <a href='https://stackoverflow.com/questions/38074678/append-existing-excel-sheet-with-new-dataframe-using-python-pandas'>stack overflow</a>